<a href="https://colab.research.google.com/github/FNSY96/TerrorismEventsPrediction/blob/master/Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python Imports**
PLEASE PUT ALL LIBRARY IMPORTS IN THIS CELL **ONLY**

In [0]:
import pandas as pd
import numpy as np
import csv
import glob
import os
import io
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [2]:
!pip3 install pandas

# Data Set Imports

In [0]:
!pip install -U -q PyDrive
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

In [4]:
# drive = GoogleDrive(gauth)
drive.mount("/content/drive", force_remount=True)
datadir  = 'drive/My Drive/GraduationProject/Data'

Mounted at /content/drive


###**GTD**

In [0]:
# !ls drive/'My Drive'/GraduationProject/Data/gtd.csv

# Pandas Data Frames
Useful link : https://youtu.be/2AFGPdNn4FM

## **GTD Data Frame**

In [6]:
gtd_df = pd.read_csv(os.path.join(datadir, 'gtd.csv'),encoding='latin-1')
print(gtd_df.columns.values)
# print(gtd_df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['eventid' 'iyear' 'imonth' 'iday' 'approxdate' 'extended' 'resolution'
 'country' 'country_txt' 'region' 'region_txt' 'provstate' 'city'
 'latitude' 'longitude' 'specificity' 'vicinity' 'location' 'summary'
 'crit1' 'crit2' 'crit3' 'doubtterr' 'alternative' 'alternative_txt'
 'multiple' 'success' 'suicide' 'attacktype1' 'attacktype1_txt'
 'attacktype2' 'attacktype2_txt' 'attacktype3' 'attacktype3_txt'
 'targtype1' 'targtype1_txt' 'targsubtype1' 'targsubtype1_txt' 'corp1'
 'target1' 'natlty1' 'natlty1_txt' 'targtype2' 'targtype2_txt'
 'targsubtype2' 'targsubtype2_txt' 'corp2' 'target2' 'natlty2'
 'natlty2_txt' 'targtype3' 'targtype3_txt' 'targsubtype3'
 'targsubtype3_txt' 'corp3' 'target3' 'natlty3' 'natlty3_txt' 'gname'
 'gsubname' 'gname2' 'gsubname2' 'gname3' 'gsubname3' 'motive'
 'guncertain1' 'guncertain2' 'guncertain3' 'individual' 'nperps'
 'nperpcap' 'claimed' 'claimmode' 'claimmode_txt' 'claim2' 'claimmode2'
 'claimmode2_txt' 'claim3' 'claimmode3' 'claimmode3_txt' 'compclaim'


In [0]:
gtd_filtered_df = gtd_df[['country','country_txt','region','region_txt','city','provstate', 'targtype1','targtype1_txt', 'iyear','imonth', 'gname', 'weaptype1', 'weaptype1_txt','attacktype1_txt','attacktype1']]
# gtd_filtered_df = gtd_filtered_df.rename(index=str, columns={"city": "city_txt", "provstate": "provstate_txt", "gname": "gname_txt"})
# print(gtd_filtered_df)
# print(gtd_filtered_df.isna().sum())

## **GECON Data Frame**

In [8]:
gecon_df = pd.read_csv(os.path.join(datadir, 'Gecon40_post_final_kh.csv'),encoding='latin-1')
# print(gecon_df.columns.values)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,19,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
gecon_filtered_df = gecon_df[['AREA', 'COUNTRY', 'DIS_LAKE','DIS_MAJOR_RIVER']]

# print(gecon_filtered_df.isna().sum())

# print(gecon_filtered_df)

## **GeoEPR Data Frame**

In [0]:
geo_df = pd.read_csv(os.path.join(datadir, 'GeoEPR-2018.1.csv'),encoding='latin-1')
# print(geo_df.columns.values)
# # statename, group
# print(geo_df)

In [0]:
geo_filtered_df = geo_df[['statename','group']]
geo_filtered_df = geo_filtered_df.rename(index=str, columns={"statename": "country_txt"})
# print(geo_filtered_df)
# print(geo_filtered_df.isna().sum())
# print(geo_filtered_df.groupby('country_txt').count())

In [0]:
# clean NaN
# geo_filtered_df = geo_filtered_df.fillna(geo_filtered_df.mean())
# attributes_NaN = ['statename', 'group', 'type']
# geo_filtered_df = remove_NaN(geo_filtered_df, attributes_NaN)

# print(geo_filtered_df)

## **Happiness World Report Data Frame**

In [0]:
happy_df_17 = pd.read_csv(os.path.join(datadir, 'Happiness2017.csv'),encoding='latin-1')
happy_df_16 = pd.read_csv(os.path.join(datadir, 'Happiness2016.csv'),encoding='latin-1')
happy_df_15 = pd.read_csv(os.path.join(datadir, 'Happiness2015.csv'),encoding='latin-1')
# happiness = [happy_df_15, happy_df_16, happy_df_17]
happy_df_17 = happy_df_17.rename(index=str, columns={"Country": "country_txt", "Happiness.Rank": "Happiness.Rank17", "Happiness.Score": "Happiness.Score17", "Freedom": "Freedom17", "Economy..GDP.per.Capita.": "Economy17"})
happy_df_16 = happy_df_16.rename(index=str, columns={"Country": "country_txt", "Happiness Rank": "Happiness.Rank16", "Happiness Score": "Happiness.Score16", "Freedom": "Freedom16", "Economy (GDP per Capita)": "Economy16"})
happy_df_15 = happy_df_15.rename(index=str, columns={"Country": "country_txt", "Happiness Rank": "Happiness.Rank15", "Happiness Score": "Happiness.Score15", "Freedom": "Freedom15", "Economy (GDP per Capita)": "Economy15"})

In [0]:
happiness_filtered = [happy_df_15[['country_txt','Happiness.Score15', 'Freedom15', 'Economy15']],
                      happy_df_16[['country_txt','Happiness.Score16', 'Freedom16', 'Economy16']], 
                      happy_df_17[['country_txt','Happiness.Score17', 'Freedom17', 'Economy17']]]

###TRIALS

In [15]:
d = {'one': pd.Series([1., 3.], index=['a', 'c']), 'two': pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd']), 'three': pd.Series(["A", "B", "A"], index=['a', 'c', 'd'])}
df = pd.DataFrame(d)
# print(df.isna().sum())
# print(df)
df = df.dropna()
print(df)
print(df.shape)
d2 = {'four': pd.Series([10., 30.], index=['a', 'c']), 'five': pd.Series([100., 200., 300., 400.], index=['a', 'b', 'c', 'd']), 'six': pd.Series(["A", "B", "A"], index=['a', 'c', 'd'])}
df2 = pd.DataFrame(d2)
df2 = df2.dropna()
print(df2)
print(df2.shape)
df3 = pd.concat([df, df2], axis = 1)
print(df3)
print(df3.shape)

# print(df.isna().sum())
# df = df.fillna(df.mean())
# mode =  df['three'].mode()[0]
# df['three'] = df.groupby('three')['three'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
# df['three']= df['three'].fillna(df['three'].value_counts().idxmax())
# print(df)
# df = pd.DataFrame(data={'location': [1, 2, 3],
#                         'coor': [(14.48847, 103.161477),
#                               (14.970084, 103.062853),
#                               (np.nan, np.nan)]})
# print(df)
# df.dropna(how='any')
# df = df[~df.coor.apply(lambda x: np.isnan(x[0]) & np.isnan(x[1]))]
# df = df[pd.DataFrame(df.coor.tolist()).notna().all(1)]
# print(df)


   one  two three
a  1.0  1.0     A
c  3.0  3.0     B
(2, 3)
   four   five six
a  10.0  100.0   A
c  30.0  300.0   B
(2, 3)
   one  two three  four   five six
a  1.0  1.0     A  10.0  100.0   A
c  3.0  3.0     B  30.0  300.0   B
(2, 6)


## MERGED DATA

### Merging

In [16]:
ethnic_count = geo_filtered_df.groupby('country_txt').count()
countries = pd.DataFrame(geo_filtered_df['country_txt'].drop_duplicates().sort_values())
ethnic_count = pd.concat([ethnic_count.reset_index(drop=True),countries.reset_index(drop=True)], axis=1)
ethnic_count = ethnic_count.rename(index=str, columns={"group":"group_count"})
print(ethnic_count.columns.values)


['group_count' 'country_txt']


In [17]:
# result = pd.concat([gtd_filtered_df, happy_filtered_df],gtd_filtered_df["country_txt"]=happy_filtered_df["Country"])
# 'Happiness.Rank' 'Happiness.Score'
gtd_merged_happiness = gtd_filtered_df
for i in range(len(happiness_filtered)):
  gtd_merged_happiness = pd.merge(gtd_merged_happiness, happiness_filtered[i],on='country_txt')
gtd_merged_happiness = pd.merge(gtd_merged_happiness, ethnic_count, on='country_txt')
print(gtd_merged_happiness.shape)
# print(gtd_merged_happiness['region_txt'].drop_duplicates())

(167161, 25)


In [18]:
print(gtd_merged_happiness.columns.values)

['country' 'country_txt' 'region' 'region_txt' 'city' 'provstate'
 'targtype1' 'targtype1_txt' 'iyear' 'imonth' 'gname' 'weaptype1'
 'weaptype1_txt' 'attacktype1_txt' 'attacktype1' 'Happiness.Score15'
 'Freedom15' 'Economy15' 'Happiness.Score16' 'Freedom16' 'Economy16'
 'Happiness.Score17' 'Freedom17' 'Economy17' 'group_count']


In [0]:
# MENA stands for Middle East and North Africa region
mena = gtd_merged_happiness.loc[gtd_merged_happiness['region_txt'] == 'Middle East & North Africa']

In [20]:
print(mena.shape)

(45846, 25)


In [21]:
print(mena['country_txt'].drop_duplicates())

16236           Jordan
19871            Egypt
23165          Lebanon
25643           Turkey
30049             Iran
61678           Israel
64027           Kuwait
78871          Algeria
91750          Morocco
114426           Syria
119326            Iraq
153137    Saudi Arabia
154671         Bahrain
157616           Yemen
Name: country_txt, dtype: object


In [22]:
# Number of distinct terrorist groups in the MENA region = 636
# print(np.unique(mena[['gname']].values))
print(np.unique(mena[['gname']]).size)

636


In [0]:
# attribute "Unknown > 27000" I think we should drop all of that
# It doesn't make sense to predict an "Unknown"
# print(mena['gname'].value_counts())
# print(mena['gname'].size)

### Filtering MENA Dataframe

In [24]:
# drop rows with nan in mena
print(mena.shape)
mena_cleaned = mena.dropna()
print(mena_cleaned.shape)

# select desired labels
mena_filtered = mena_cleaned[(mena_cleaned['gname'] == "Islamic State of Iraq and the Levant (ISIL)") | (mena_cleaned['gname'] == "Kurdistan Workers' Party (PKK)") | (mena_cleaned['gname'] == "Houthi extremists (Ansar Allah)") | (mena_cleaned['gname'] == "Al-Qaida in the Arabian Peninsula (AQAP)") | (mena_cleaned['gname'] == "Al-Qaida in Iraq")]

print(mena_filtered['gname'].value_counts())
print(mena_filtered['gname'].size)

(45846, 25)
(45760, 25)
Islamic State of Iraq and the Levant (ISIL)    5553
Kurdistan Workers' Party (PKK)                 2137
Houthi extremists (Ansar Allah)                1062
Al-Qaida in the Arabian Peninsula (AQAP)       1016
Al-Qaida in Iraq                                636
Name: gname, dtype: int64
10404


# Training

In [0]:
# labels = np.unique(mena_filtered[['gname']])
# labels = pd.DataFrame(labels)
# print(labels)
# print(type(labels))


In [25]:
label_encoder = preprocessing.LabelEncoder()
string_attributes = ['city', 'provstate', 'gname']
mena_final = mena_filtered
mena_final = mena_final.drop(["gname", "provstate", "city"], axis=1)
print(mena_final.shape)
print(mena_final.columns)

for i in range(len(string_attributes)):
  attribute_name = string_attributes[i]
  attribute_list = mena_filtered[attribute_name]
  attribute_encoded = label_encoder.fit_transform(attribute_list)
  value_zipped_encoded_list = list(zip(attribute_list,attribute_encoded))
  value_zipped_encoded_df = pd.DataFrame(value_zipped_encoded_list)
  modified_attribute_name = attribute_name + '_numerical'
  value_zipped_encoded_df.columns = [attribute_name, modified_attribute_name]
  zipped_df = pd.DataFrame(value_zipped_encoded_df)
  print(zipped_df.shape)
  print(mena_final.shape)
  mena_final.reset_index(drop=True, inplace=True)
  zipped_df.reset_index(drop=True, inplace=True)
  mena_final = pd.concat([mena_final, zipped_df], axis = 1)
  print("after concat")
  print(mena_final.shape)
  print("*************")

print(mena_final.shape) 
# print(mena_final.isna().sum())
mena_final = mena_final.dropna()
print(mena_final.shape)
print(mena_final.columns)
# zipped_df = pd.DataFrame(value_zipped_encoded_df)
# mena_final = pd.concat([zipped_df, mena_filtered], axis = 1)
# print(zipped_df.columns.values)
print(mena_final)
# print(mena_final.columns.values)

(10404, 22)
Index(['country', 'country_txt', 'region', 'region_txt', 'targtype1',
       'targtype1_txt', 'iyear', 'imonth', 'weaptype1', 'weaptype1_txt',
       'attacktype1_txt', 'attacktype1', 'Happiness.Score15', 'Freedom15',
       'Economy15', 'Happiness.Score16', 'Freedom16', 'Economy16',
       'Happiness.Score17', 'Freedom17', 'Economy17', 'group_count'],
      dtype='object')
(10404, 2)
(10404, 22)
after concat
(10404, 24)
*************
(10404, 2)
(10404, 24)
after concat
(10404, 26)
*************
(10404, 2)
(10404, 26)
after concat
(10404, 28)
*************
(10404, 28)
(10404, 28)
Index(['country', 'country_txt', 'region', 'region_txt', 'targtype1',
       'targtype1_txt', 'iyear', 'imonth', 'weaptype1', 'weaptype1_txt',
       'attacktype1_txt', 'attacktype1', 'Happiness.Score15', 'Freedom15',
       'Economy15', 'Happiness.Score16', 'Freedom16', 'Economy16',
       'Happiness.Score17', 'Freedom17', 'Economy17', 'group_count', 'city',
       'city_numerical', 'provstate', '

### Divide Data into Train & Test

In [59]:
# divide dataset into train and test
mena_classification = mena_final.drop(["gname", "country_txt", "region_txt", "targtype1_txt", "provstate", "city", "weaptype1_txt", "attacktype1_txt", "imonth", "region"], axis=1)
mena_classification = mena_classification.drop(['Happiness.Score15', 'Freedom15', 'Economy15','Happiness.Score16', 'Freedom16', 'Economy16','Happiness.Score17', 'Freedom17', 'Economy17'], axis=1)

mena_classification = mena_classification[mena_classification['iyear'] >= 2000]
print(mena_classification.shape)

iyear = 2017

mena_split_train = mena_classification[mena_classification['iyear'] < iyear]
mena_split_test = mena_classification[mena_classification['iyear'] >= iyear]

mena_split_train = mena_split_train.drop("iyear", axis = 1)
mena_split_test = mena_split_test.drop("iyear", axis = 1)


labels_train = mena_split_train['gname_numerical']
labels_test = mena_split_test['gname_numerical']

data_train = mena_split_train.loc[:, mena_split_train.columns != 'gname_numerical']
data_test = mena_split_test.loc[:, mena_split_test.columns != 'gname_numerical']

# print(labels)
# print(trained_data)

# mena_train = trained_data.copy()
# mena_test = trained_data.copy()

# mena_train = mena_train[mena_train['iyear'] < 2017]
# mena_test = mena_test[mena_test['iyear'] >= 2017]

print(mena_split_train.shape)
print(data_train.columns.values)
print(mena_split_test.shape)
print(data_test.columns.values)

(9490, 8)
(7842, 7)
['country' 'targtype1' 'weaptype1' 'attacktype1' 'city_numerical'
 'provstate_numerical']
(1648, 7)
['country' 'targtype1' 'weaptype1' 'attacktype1' 'city_numerical'
 'provstate_numerical']


In [47]:
aa = pd.DataFrame(labels_train)
print(np.unique(aa))

bb = pd.DataFrame(labels_test)
print(np.unique(bb))

[0 1 2 3 4]
[1 2 3 4]


In [0]:
ll = np.array(labels_test)

print(type(ll))
for i in range(len(ll)):
  print(ll[i])

###KNN Classifier

In [54]:
scaler = StandardScaler()  
scaler.fit(data_train)

data_train = scaler.transform(data_train)  
data_test = scaler.transform(data_test)  

acc = 0

for i in range(3,1000):

  # Create KNN Classifier
  knn = KNeighborsClassifier(n_neighbors = i)

  # Train the model using the training sets
  knn.fit(data_train, labels_train)

  # Predict the response for test dataset
  y_pred = knn.predict(data_test)

  # Model Accuracy, how often is the classifier correct?
  curr_acc =  metrics.accuracy_score(labels_test, y_pred)
  if curr_acc > acc:
    acc = curr_acc
    print("Accuracy:", acc)
    print(confusion_matrix(labels_test[0:in_test], y_pred))  
    print(classification_report(labels_test[0:in_test], y_pred))
    print(i)
    

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.7730582524271845
[[  0   0   0   0   0]
 [  0  38   5   1   0]
 [  0  41 109   8   0]
 [284  10   4 986   7]
 [  1   0   0  13 141]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.43      0.86      0.57        44
           2       0.92      0.69      0.79       158
           3       0.98      0.76      0.86      1291
           4       0.95      0.91      0.93       155

   micro avg       0.77      0.77      0.77      1648
   macro avg       0.66      0.65      0.63      1648
weighted avg       0.96      0.77      0.85      1648

3
Accuracy: 0.8610436893203883
[[   0    0    0    0    0]
 [   0   40    4    0    0]
 [   2   32  120    4    0]
 [ 147   14    6 1117    7]
 [   1    0    0   12  142]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.47      0.91      0.62        44
           2       0.92      0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.8628640776699029
[[   0    0    0    0    0]
 [   0   38    6    0    0]
 [   0   37  110   11    0]
 [ 131    7   14 1132    7]
 [   1    0    0   12  142]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.46      0.86      0.60        44
           2       0.85      0.70      0.76       158
           3       0.98      0.88      0.93      1291
           4       0.95      0.92      0.93       155

   micro avg       0.86      0.86      0.86      1648
   macro avg       0.65      0.67      0.65      1648
weighted avg       0.95      0.86      0.90      1648

7
Accuracy: 0.8938106796116505
[[   0    0    0    0    0]
 [   0   37    6    1    0]
 [   0   44  103   11    0]
 [  74    7   12 1191    7]
 [   1    0    0   12  142]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.42      0.84      0.56        44
      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


...

Accuracy: 0.904126213592233
[[   0    0    0    0    0]
 [   0   37    7    0    0]
 [   0   35  109   14    0]
 [  18   26   34 1207    6]
 [   1    0    0   17  137]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.38      0.84      0.52        44
           2       0.73      0.69      0.71       158
           3       0.97      0.93      0.95      1291
           4       0.96      0.88      0.92       155

   micro avg       0.90      0.90      0.90      1648
   macro avg       0.61      0.67      0.62      1648
weighted avg       0.93      0.90      0.92      1648

49
Accuracy: 0.9059466019417476
[[   0    0    0    0    0]
 [   0   37    7    0    0]
 [   0   35  109   14    0]
 [  15   26   34 1210    6]
 [   1    0    0   17  137]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.38      0.84      0.52        44
      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.9114077669902912
[[   0    0    0    0    0]
 [   0   36    8    0    0]
 [   0   31  112   15    0]
 [   7   20   41 1217    6]
 [   0    0    0   18  137]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.41      0.82      0.55        44
           2       0.70      0.71      0.70       158
           3       0.97      0.94      0.96      1291
           4       0.96      0.88      0.92       155

   micro avg       0.91      0.91      0.91      1648
   macro avg       0.61      0.67      0.63      1648
weighted avg       0.93      0.91      0.92      1648

63


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


KeyboardInterrupt: ignored

###Naïve Bayes

In [60]:
# clf.fit(X_train[0:index], Y_train
index = len(labels_train);
in_test = len(labels_test)

# Create a  Naive Bayes Classifier
clf = GaussianNB()

# Train the model using the training sets
clf.fit(data_train[0:index], labels_train[0:index])

# Predict the response for test dataset
y_pred = clf.predict(data_test[0:in_test])

print("Accuracy:", metrics.accuracy_score(labels_test[0:in_test], y_pred))
print(confusion_matrix(labels_test[0:in_test], y_pred))  
print(classification_report(labels_test[0:in_test], y_pred))

Accuracy: 0.3446601941747573
[[  0   0   0   0   0]
 [  0  41   3   0   0]
 [  0  77  52  29   0]
 [817   0  28 347  99]
 [  7   0  16   4 128]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.35      0.93      0.51        44
           2       0.53      0.33      0.40       158
           3       0.91      0.27      0.42      1291
           4       0.56      0.83      0.67       155

   micro avg       0.34      0.34      0.34      1648
   macro avg       0.47      0.47      0.40      1648
weighted avg       0.83      0.34      0.44      1648



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Random Forests

In [62]:
acc = 0
for i in range(3,1000):
  # Create a Gaussian Classifier
  clf = RandomForestClassifier(n_estimators = i)

  # Train the model using the training sets y_pred=clf.predict(X_test)
  clf.fit(data_train, labels_train)    
  y_pred=clf.predict(data_test)
  
  curr_acc =  metrics.accuracy_score(labels_test, y_pred)
  if curr_acc > acc:
    acc = curr_acc
    print("Accuracy:", acc)
    print(confusion_matrix(labels_test[0:in_test], y_pred))  
    print(classification_report(labels_test[0:in_test], y_pred))
    print(i)

Accuracy: 0.9205097087378641
[[   0    0    0    0    0]
 [   0   42    2    0    0]
 [   0   26  128    4    0]
 [  79    0    0 1206    6]
 [   0    0    0   14  141]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.62      0.95      0.75        44
           2       0.98      0.81      0.89       158
           3       0.99      0.93      0.96      1291
           4       0.96      0.91      0.93       155

   micro avg       0.92      0.92      0.92      1648
   macro avg       0.71      0.72      0.71      1648
weighted avg       0.97      0.92      0.94      1648

3
Accuracy: 0.9296116504854369
[[   0    0    0    0    0]
 [   0   40    4    0    0]
 [   0   27  130    1    0]
 [  63    0    0 1220    8]
 [   0    0    0   13  142]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.60      0.91      0.72        44
      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.9387135922330098
[[   0    0    0    0    0]
 [   0   42    2    0    0]
 [   0   32  126    0    0]
 [  47    0    0 1237    7]
 [   0    0    0   13  142]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.57      0.95      0.71        44
           2       0.98      0.80      0.88       158
           3       0.99      0.96      0.97      1291
           4       0.95      0.92      0.93       155

   micro avg       0.94      0.94      0.94      1648
   macro avg       0.70      0.73      0.70      1648
weighted avg       0.97      0.94      0.95      1648

13


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.941747572815534
[[   0    0    0    0    0]
 [   0   42    2    0    0]
 [   0   26  131    1    0]
 [  48    0    0 1237    6]
 [   0    0    0   13  142]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.62      0.95      0.75        44
           2       0.98      0.83      0.90       158
           3       0.99      0.96      0.97      1291
           4       0.96      0.92      0.94       155

   micro avg       0.94      0.94      0.94      1648
   macro avg       0.71      0.73      0.71      1648
weighted avg       0.98      0.94      0.96      1648

17


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.945995145631068
[[   0    0    0    0    0]
 [   0   43    1    0    0]
 [   0   22  132    4    0]
 [  42    0    0 1242    7]
 [   0    0    0   13  142]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.66      0.98      0.79        44
           2       0.99      0.84      0.91       158
           3       0.99      0.96      0.97      1291
           4       0.95      0.92      0.93       155

   micro avg       0.95      0.95      0.95      1648
   macro avg       0.72      0.74      0.72      1648
weighted avg       0.98      0.95      0.96      1648

35


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


KeyboardInterrupt: ignored

###Decision tree


In [57]:
clf = tree.DecisionTreeClassifier()
clf.fit(data_train, labels_train)    #Train the model using the training sets y_pred=clf.predict(X_test)
y_pred=clf.predict(data_test)

print("Accuracy:", metrics.accuracy_score(labels_test, y_pred))
print(confusion_matrix(labels_test, y_pred))  
print(classification_report(labels_test, y_pred))

# CART (Classification and Regression Trees) is very similar to C4.5, 
# but it differs in that it supports numerical target variables (regression) and does not compute rule sets. 
# CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.
# scikit-learn uses an optimised version of the CART algorithm.

Accuracy: 0.9089805825242718
[[   0    0    0    0    0]
 [   0   41    3    0    0]
 [   0   29  129    0    0]
 [  95    0    0 1189    7]
 [   1    0    0   15  139]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.59      0.93      0.72        44
           2       0.98      0.82      0.89       158
           3       0.99      0.92      0.95      1291
           4       0.95      0.90      0.92       155

   micro avg       0.91      0.91      0.91      1648
   macro avg       0.70      0.71      0.70      1648
weighted avg       0.97      0.91      0.94      1648



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


###SVM

In [58]:
index = len(labels_train);
in_test = len(labels_test);
#Create a svm Classifier
clf = svm.SVC(kernel = 'linear') # Linear Kernel

#Train the model using the training sets
clf.fit(data_train[0:index], labels_train[0:index])

#Predict the response for test dataset
y_pred = clf.predict(data_test[0:in_test])

print("Accuracy:", metrics.accuracy_score(labels_test[0:in_test], y_pred))
print(confusion_matrix(labels_test[0:in_test], y_pred))  
print(classification_report(labels_test[0:in_test], y_pred))

Accuracy: 0.9326456310679612
[[  34   10    0    0]
 [  52   77   29    0]
 [   0    0 1284    7]
 [   0    0   13  142]]
              precision    recall  f1-score   support

           1       0.40      0.77      0.52        44
           2       0.89      0.49      0.63       158
           3       0.97      0.99      0.98      1291
           4       0.95      0.92      0.93       155

   micro avg       0.93      0.93      0.93      1648
   macro avg       0.80      0.79      0.77      1648
weighted avg       0.94      0.93      0.93      1648

